In [20]:
# Branch-and-Bound optimization method for integer programming
# Engineering Optimization: Theory and Practice, Fourth Edition Singiresu S. Rao
# This code can be used to solve all-integer and mixed integer linear programming problems
# 09 Oct 2024, Written by Seshu Damarla

import numpy as np
import math
from scipy.optimize import linprog

# Checking if the solution is an integer solution
def is_integer(x):
    return np.all(np.mod(x, 1) == 0)

# Branch-and-bound optimization method
def branch_and_bound(C, A, b, lb, ub, IsMax, depth, integer_variables):
    if IsMax:
        res = linprog(-C, A_ub=A, b_ub=b, bounds=list(zip(lb, ub)))
    else:
        res = linprog(C, A_ub=A, b_ub=b, bounds=list(zip(lb, ub)))

    if not res.success:
        if IsMax:
            print("Problem appears to be infeasible")
            return [], -np.inf, depth
        else:
            print("Problem appears to be unbounded")
            return [], np.inf, depth

    x_opt = res.x
    f_opt = res.fun

    # Check if the optimal solution satisfies all integer constraints
    constr_var_satisfied = all(is_integer(x_opt[idx]) for idx, char in enumerate(integer_variables) if char)

    if constr_var_satisfied:
        return x_opt, -f_opt if IsMax else f_opt, depth

    # Create new bounds for branching
    lb_1, ub_1 = lb.copy(), ub.copy()
    lb_2, ub_2 = lb.copy(), ub.copy()

    chk_indx = next((idx for idx, char in enumerate(integer_variables) if char and not is_integer(x_opt[idx])), None)
    if chk_indx is not None:
        # Branching on the variable that is not integer
        ub_1[chk_indx] = math.floor(x_opt[chk_indx])  # Tighten upper bound
        lb_2[chk_indx] = math.ceil(x_opt[chk_indx])   # Tighten lower bound

        print("Creating two optimization problems")
        print('Solving the first optimization problem')
        x_opt_1, f_opt_1, depth_1 = branch_and_bound(C, A, b, lb_1, ub_1, IsMax, depth + 1, integer_variables)
        print('Solving the second optimization problem')
        x_opt_2, f_opt_2, depth_2 = branch_and_bound(C, A, b, lb_2, ub_2, IsMax, depth + 1, integer_variables)

        if IsMax:
            return (x_opt_1, -f_opt_1, depth_1) if f_opt_1 > f_opt_2 else (x_opt_2, -f_opt_2, depth_2)
        else:
            return (x_opt_1, f_opt_1, depth_1) if f_opt_1 < f_opt_2 else (x_opt_2, f_opt_2, depth_2)

# Example parameters
if __name__ == "__main__":
    """C = np.array([4, 3, 1])
    A = np.array([[3, 2, 1], [2, 1, 2]])
    b = np.array([7, 11])
    lb = [0, 0, 0]
    ub = [10**8, 10**8, 10**8]
    integer_variables = [False, True, True]"""

    C = np.array([3, 4])
    A = np.array([[7, 11], [3, -1]])
    b = np.array([88, 12])
    lb = [0, 0]
    ub = [10**8, 10**8]
    integer_variables = [True, True]

    IsMax = True
    depth = 0

    x_opt, f_opt, depth = branch_and_bound(C, A, b, lb, ub, IsMax, depth, integer_variables)
    print("The optimal solution is", x_opt)
    print("The optimal value is", f_opt)
    print("The depth of the tree is", depth)


Creating two optimization problems
Solving the first optimization problem
Creating two optimization problems
Solving the first optimization problem
Solving the second optimization problem
Creating two optimization problems
Solving the first optimization problem
Creating two optimization problems
Solving the first optimization problem
Solving the second optimization problem
Creating two optimization problems
Solving the first optimization problem
Creating two optimization problems
Solving the first optimization problem
Solving the second optimization problem
Creating two optimization problems
Solving the first optimization problem
Creating two optimization problems
Solving the first optimization problem
Solving the second optimization problem
Solving the second optimization problem
Problem appears to be infeasible
Solving the second optimization problem
Problem appears to be infeasible
Solving the second optimization problem
Problem appears to be infeasible
Solving the second optimizati